#### Facebook
* ChinaTimes 188311137478
* udn 241284961029
* ltn 394896373929368
* AppleDaily 232633627068

In [17]:
import requests
import json
import os
import datetime
import time
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import urllib
import time

###Setting variables

In [18]:
def get_fb_token(app_id, app_secret):           
    payload = {'grant_type': 'client_credentials', 
               'client_id': app_id, 
               'client_secret': app_secret}
    file = requests.post('https://graph.facebook.com/oauth/access_token?', 
                         params = payload)
    result = file.text.split("=")[1]
    return result

def query_url(pageid, q, token):
    return "https://graph.facebook.com/v2.6/%s?%s&access_token=%s"%(pageid, q, token)
#     
pageid = '188311137478' #ChinaTimes 
# pageid = '241284961029' #udn 
# pageid = '394896373929368' #ltn 
# pageid = '232633627068' #AppleDaily 
app_id = '1577374979150890'
app_secret = '6408b3bcaabd6a87d65200976f76a387'

token = 'EAACEdEose0cBABX10l1ZC7RIhSY1NGIrbHKL3oCQAdkZCJGbBl5YDlBZB4es4e9GOhJqXZCzXZBPHSXqw0a3XSkQZBaMtpOOItGF2H5xEzfEIa7FS0BhAiJImIcH6a2nLgtk8vZBf7sPzb6ZBs7LLcfpypp4bsM3ZBUm8bbWd6mdhT0Te1TmAk4v9'
token = get_fb_token(app_id, app_secret)

###Getting post

In [15]:
directory = "data/%s/"%pageid
if not os.path.exists(directory):
    os.makedirs(directory)
directory = directory + datetime.datetime.now().strftime('%y%m%d%H%M%S') + '/'
if not os.path.exists(directory):
    os.makedirs(directory)


url = query_url(pageid, 'fields=posts{shares,message,permalink_url,created_time}', token)
results = requests.get(url)
data = json.loads(results.text)
print data['posts']['data'][0]
# # posts = data['posts']['data']
# # print data['posts']['paging']['next']
# # url = data['posts']['paging']['next']

# # pids = [p['id'] for p in data['posts']['data']]
# while True:
#     results = requests.get(url)
#     data = json.loads(results.text)
#     if len(data['data']) == 0:
#         break    
#     posts += data['data']
# #     print data['paging']['next']
#     url = data['paging']['next']
#     print 'Total post:%d'%len(posts)

# # print len(posts)
# fname = directory + 'posts.json'
# json.dump(posts, open(fname, 'w'))

{u'created_time': u'2017-01-16T04:39:00+0000', u'message': u'\u9084\u5305\u819c\u5594 ^_<  #\u5c3e\u7259', u'permalink_url': u'https://www.facebook.com/myudn/posts/10155366213971030', u'id': u'241284961029_10155366213971030'}


###Get comments of each comment

In [ ]:
def getCofC(comments, fname, onlyComment=True): # getting comment of comment
    allC = []
    for i, c in enumerate(comments):
        if onlyComment:
            if c['comment_count'] != 0:
                print "C%d[%s]:\tCC:%d\t"%(i, c['id'],c['comment_count'])
                url = query_url(c['id'], 'fields=comments.limit(%d)'%c['comment_count'], token)

                results = requests.get(url)
                data = json.loads(results.text)
                allC.append({c['id']:{'cc':[] if 'comments' not in data else data['comments']['data'], 
                                      'like':[] if 'likes' not in data else data['likes']['data']
                                     }})
        else:
            if c['comment_count'] != 0 or c['like_count'] != 0:
                print "C%d[%s]:\tCC:%d\tLike:%d"%(i, c['id'],c['comment_count'],c['like_count'])
                url = query_url(c['id'], 'fields=comments.limit(%d),likes.limit(%d)'%(c['comment_count'], c['like_count']), token)

                results = requests.get(url)
                data = json.loads(results.text)
                allC.append({c['id']:{'cc':[] if 'comments' not in data else data['comments']['data'], 
                                      'like':[] if 'likes' not in data else data['likes']['data']
                                     }})
    with open(fname+'.test', 'w') as fout:
        json.dump(allC, fout)
    print "...Succesfully Save", fname

###Get comments of each posts

In [ ]:
start = time.time()
DEBUG = False
amount = 50 if DEBUG else 500
print "Length of posts: %d"%len(posts)

for i, p in enumerate(posts):
#     if i < 218:
#         continue
#     if i not in [23]:
#         continue
    
#     url = query_url(p['id'], 'fields=comments.limit(500){like_count,comment_count,created_time,from,message}', token)
    url = query_url(p['id'], 'fields=comments.limit(%d){comment_count,like_count,created_time,from,message}'%amount, token)
    results = requests.get(url)
    data = json.loads(results.text)
    if len(data.keys()) == 1:
        print "ERROR:CANNOT GET THE POST:", data
        continue
    data = data['comments']
    comments = data['data']
    if 'next' in data['paging']:
        while True:
            url = data['paging']['next']
            results = requests.get(url)
            data = json.loads(results.text)
            try:
                comments += data['data']
            except:
                time.sleep(3)
                print "ERROR: time sleep 3 seconds"
                print data.keys()
                comments  += data['data']
            if 'next' not in data['paging']:
                break
    print 'PostID[%d] %s (comments:%d) time:%d'%(i, p['id'], len(comments), time.time()-start)
    print p['permalink_url']
    
    getCofC(comments, directory + 'c2cment%d-%s.json'%(i, p['id']))
    fname = directory + 'comment%d-%s.json'%(i, p['id'])
    with open(fname, 'w') as fout:
        json.dump(comments, fout)
#####################################################################################
# Getting comments of comments
    maxcc = max(c['comment_count'] for c in comments)
    if DEBUG: print('maxcc:%d'%maxcc)
    url = query_url(p['id'], 'fields=comments.limit(%d){comments.limit(%d)}'%(amount, maxcc), token)
    results = requests.get(url)
    data = json.loads(results.text)
    if len(data.keys()) == 1:
        print "ERROR:CANNOT GET THE POST:", data
        continue
    data = data['comments']
    c2cs = data['data']
    if 'next' in data['paging']:
        while True:
            url = data['paging']['next']
            results = requests.get(url)
            data = json.loads(results.text)
            try:
                c2cs += data['data']
            except:
                time.sleep(3)
                print "ERROR: time sleep 3 seconds"
                print data.keys()
                c2cs  += data['data']
            if 'next' not in data['paging']:
                break
    print 'PostID[%d] %s (c2cs:%d) time:%d'%(i, p['id'], len(c2cs), time.time()-start)
    print p['permalink_url']
    fname = directory + 'c2cment%d-%s.json'%(i, p['id'])
    with open(fname, 'w') as fout:
        json.dump(c2cs, fout)
